In [1]:
# 下載資料
# !gdown --id '14CqX3OfY9aUbhGp4OpdSHLvq2321fUB7' --output data.zip
# 解壓縮
# !unzip data.zip

In [2]:
import os
# 讀取 label.csv
import pandas as pd
# 讀取圖片
from PIL import Image
import numpy as np

import torch
# Loss function
import torch.nn.functional as F
# 讀取資料
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
# 載入預訓練的模型
import torchvision.models as models
# 將資料轉換成符合預訓練模型的形式
import torchvision.transforms as transforms
# 顯示圖片
import matplotlib.pyplot as plt

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")

In [3]:
# 實作一個繼承 torch.utils.data.Dataset 的 Class 來讀取圖片
class Adverdataset(Dataset):
    def __init__(self, root, label, transforms):
        # 圖片所在的資料夾
        self.root = root
        # 由 main function 傳入的 label
        self.label = torch.from_numpy(label).long()
        # 由 Attacker 傳入的 transforms 將輸入的圖片轉換成符合預訓練模型的形式
        self.transforms = transforms
        # 圖片檔案名稱的 list
        self.fnames = []

        for i in range(200):
            self.fnames.append("{:03d}".format(i))

    def __getitem__(self, idx):
        # 利用路徑讀取圖片
        img = Image.open(os.path.join(self.root, self.fnames[idx] + '.png'))
        # 將輸入的圖片轉換成符合預訓練模型的形式
        img = self.transforms(img)
        # 圖片相對應的 label
        label = self.label[idx]
        return img, label
    
    def __len__(self):
        # 由於已知這次的資料總共有 200 張圖片 所以回傳 200
        return 200

In [4]:
class Attacker:
    def __init__(self, img_dir, label):
        self.model = models.vgg16(pretrained=True)
        self.model.to(DEVICE)
        self.model.eval()
        self.resize_factor = 300
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]        
        rnd = np.random.randint(224, self.resize_factor)
        w_rem = self.resize_factor - rnd
        pad_left = np.random.randint(0, w_rem)
        pad_right = w_rem - pad_left
        h_rem = self.resize_factor - rnd
        pad_top = np.random.randint(0, h_rem)
        pad_bottom = h_rem - pad_left
        
        basic_transform = transforms.Compose([
            transforms.Resize((224, 224), interpolation=3),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std, inplace=False)
        ])
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomApply([
                transforms.Resize(rnd),
                transforms.Pad((pad_left, pad_top, pad_right, pad_bottom))
            ], 0.5),
            transforms.Resize((224, 224), interpolation=3),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std, inplace=False)
        ])
        # 利用 Adverdataset 這個 class 讀取資料
        self.dataset = Adverdataset('./data/images', label, basic_transform)
        
        self.loader = torch.utils.data.DataLoader(
                self.dataset,
                batch_size = 1,
                shuffle = False)

    # DI-2-FGSM
    def fgsm(self, epsilon, data, target):
        transformed = self.transform(data.cpu()).to(DEVICE)
        transformed = transformed.reshape((1, 3, 224, 224))
        transformed.requires_grad = True
        output = self.model(transformed)
        pred = output.max(1, keepdim=True)[1]
        if pred.item() != target.item():
            return data
        loss = F.nll_loss(output, target)
        self.model.zero_grad()
        loss.backward()
        data_grad = transformed.grad.data / torch.mean(transformed.grad.abs(), (1, 2, 3), keepdim=True)
        data += data_grad.reshape(3, 224, 224) * epsilon
        return data
    
    def attack(self, epsilon):
        adv_results = []
        failed = 0
        for (data, target) in self.dataset:
            data, target = data.to(DEVICE), target.to(DEVICE)
            target = target.reshape((1))
            adv_data = data.clone().detach()
            
            for itr in range(4):
                adv_data = self.fgsm(epsilon, adv_data, target)
            adv_data = adv_data.reshape((1, 3, 224, 224))
            output = self.model(adv_data)
            pred = output.max(1, keepdim=True)[1]
            if pred.item() == target.item():
                failed += 1
            adv_ex = adv_data * torch.tensor(self.std, device=DEVICE).view(3, 1, 1) + torch.tensor(self.mean, device=DEVICE).view(3, 1, 1)
            adv_ex = adv_ex.squeeze().detach().cpu().numpy()
            np.clip(adv_ex, 0, 1, out=adv_ex)
            data_raw = data * torch.tensor(self.std, device=DEVICE).view(3, 1, 1) + torch.tensor(self.mean, device=DEVICE).view(3, 1, 1)
            data_raw = data_raw.squeeze().detach().cpu().numpy()
            adv_results.append( (target.item(), pred.item(), data_raw, adv_ex) )
        final_acc = failed / len(self.loader)
        print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, failed, len(self.loader), final_acc))
        return adv_results, final_acc


In [ ]:
# 讀入圖片相對應的 label
df = pd.read_csv("./data/labels.csv")
df = df.loc[:, 'TrueLabel'].to_numpy()
label_name = pd.read_csv("./data/categories.csv")
label_name = label_name.loc[:, 'CategoryName'].to_numpy()
# new 一個 Attacker class
attacker = Attacker('./data/images', df)
# 要嘗試的 epsilon
epsilons = [1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1]

accuracies, examples = [], []
results = []

# 進行攻擊 並存起正確率和攻擊成功的圖片
for eps in epsilons:
    res, acc = attacker.attack(eps)
    accuracies.append(acc)
    examples.append(res[:5])
    results.append(res)
    L_inf = 0
    for img in res:
        L_inf += np.linalg.norm((img[2] - img[3]).reshape(224 * 224 * 3), ord=np.inf)
    L_inf /= 200
    L_inf *= 255
    print(f'L-inf = {L_inf}\n')

Epsilon: 0.001	Test Accuracy = 173 / 200 = 0.865
L-inf = 0.08358348794281482

Epsilon: 0.005	Test Accuracy = 173 / 200 = 0.865
L-inf = 0.452865931391716



In [ ]:
%matplotlib inline
# 顯示圖片
cnt = 0
plt.figure(figsize=(30, 30))
for i in range(len(epsilons)):
    try:
        os.mkdir(f'results/{epsilons[i]}')
    except FileExistsError:
        pass
    for j, img in enumerate(results[i]):
        img = img[3] * 255
        img = img.astype('uint8')
        img = np.transpose(img, (1, 2, 0))
        img = Image.fromarray(img, 'RGB')
        img.save(f'results/{epsilons[i]}/%03d.png' % j)

    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]) * 2,cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig, adv, orig_img, ex = examples[i][j]
        # plt.title("{} -> {}".format(orig, adv))
        plt.title("original: {}".format(label_name[orig].split(',')[0]))
        orig_img = np.transpose(orig_img, (1, 2, 0))
        plt.imshow(orig_img)
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]) * 2,cnt)
        plt.title("adversarial: {}".format(label_name[adv].split(',')[0]))
        ex = np.transpose(ex, (1, 2, 0))
        plt.imshow(ex)
plt.tight_layout()
plt.show()

In [ ]:
torch.cuda.empty_cache()